Jupyter Notebook used to calculate the 3D Pair Distribution Function from large reciprocal space maps (diffraction patterns) obtained at QM2 at the CHESS Synchrotron Source

In [ ]:
## import modules
from nexusformat.nexus import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
### function to interpolate over the punched out regions

def pad(data):
    bad_indexes = np.isnan(data)
    good_indexes = np.logical_not(bad_indexes)
    good_data = data[good_indexes]
    interpolated = np.interp(bad_indexes.nonzero()[0], good_indexes.nonzero()[0], 
    good_data)
    data[bad_indexes] = interpolated
    return data

In [ ]:
#load diffraction pattern, produced by indexing scripts
a=nxload('/data3/2021/2021_10_CHESS_3088D/ind_jg2364/index_new_nm/bscco_attempt_optimized.nxs')
print(a.tree)

In [ ]:
### setup reciprocal space 
H=np.array(a.entry.data.H)
K=np.array(a.entry.data.K)
L=np.array(a.entry.data.L)

In [ ]:
### match array indices with reciprocal lattice units
def fi(H,value):
    for n in range(len(H)):
        if H[n]>value:
            break
    return n-1

In [ ]:
### Visualize the diffraction pattern
%matplotlib notebook

plt.imshow(np.rot90(np.log(a.entry.data.counts[fi(H,2.0),:,:])),extent=[K[0],K[-1],L[0],L[-1]],  aspect='auto')

plt.colorbar()

plt.figure()

plt.imshow(np.rot90(np.log(a.entry.data.counts[:,:,fi(L,10)])),extent=[H[0],H[-1],K[0],K[-1]],  aspect='auto')
plt.colorbar()


plt.figure()

plt.imshow(np.rot90(np.log(a.entry.data.counts[:,fi(K,2.0),:])),extent=[H[0],H[-1],L[0],L[-1]],  aspect='auto')
plt.colorbar()


In [ ]:
### Convert the 3D array to floats
data_set = np.array(a.entry.data.counts[:,:,:].astype('float32'))
data_set_nan = np.array(a.entry.data.counts[:,:,:].astype('float32'))
print(data_set.shape)


In [ ]:
### Punch out certain diffraction features. Many different variations of this can be implemented (e.g. remove diffuse features or remove Bragg peaks)
#### Here, high intensity features are removed and interpolated over
data_punched = data_set
data_punched[np.where(data_punched>10**(2.))]=np.nan

In [ ]:
### alternatively, one can remove all the diffuse scattering
data_punched = data_set
data_punched[np.where(data_punched<10**(4.5))]=0

data_punched[fi(H,-5):fi(H,-4.1),:,:]=0
data_punched[fi(H,-3.9):fi(H,-3.1),:,:]=0
data_punched[fi(H,-2.9):fi(H,-2.1),:,:]=0
data_punched[fi(H,-1.9):fi(H,-1.1),:,:]=0
data_punched[fi(H,-0.9):fi(H,-0.1),:,:]=0
data_punched[fi(H,0.1):fi(H,0.9),:,:]=0
data_punched[fi(H,1.1):fi(H,1.9),:,:]=0
data_punched[fi(H,2.1):fi(H,2.9),:,:]=0
data_punched[fi(H,3.1):fi(H,3.9),:,:]=0
data_punched[fi(H,4.1):fi(H,5),:,:]=0

In [ ]:
### visualize the augmented diffraction data
%matplotlib notebook

plt.imshow(np.rot90(np.log10(data_punched[fi(H,0.01),:,:])),extent=[K[0],K[-1],L[0],L[-1]],  aspect='auto')
plt.colorbar()

plt.figure()

plt.imshow(np.rot90(np.log10(data_punched[:,:,fi(L,10)])),extent=[H[0],H[-1],K[0],K[-1]],  aspect='auto')
plt.colorbar()

plt.figure()

plt.imshow(np.rot90(np.log10(data_punched[:,fi(K,0.0),:])),extent=[H[0],H[-1],L[0],L[-1]],  aspect='auto')
plt.colorbar()

In [ ]:
### interpolate over NANs, only used when replacing high intensity feautres with NANs
data_set_toInterp = data_punched.copy()

interped = np.apply_along_axis(pad, 0, data_set_toInterp)

In [ ]:
### visualize the augmented diffraction data, interpolated over Nan regions
%matplotlib notebook

plt.imshow(np.rot90(np.log(interped[fi(H,0.01),:,:])),extent=[K[0],K[-1],L[0],L[-1]],  aspect='auto')
plt.colorbar()

plt.figure()

plt.imshow(np.rot90(np.log(interped[:,:,fi(L,10)])),extent=[H[0],H[-1],K[0],K[-1]],  aspect='auto')
plt.colorbar()


plt.figure()

plt.imshow(np.rot90(np.log(interped[:,fi(K,0.0),:])),extent=[H[0],H[-1],L[0],L[-1]],  aspect='auto')
plt.colorbar()

In [ ]:
### Symmetrize the Diffraction Pattern
### A diffraction pattern is always symmetric according to Friedel's Law. For the CHESS data we only have a portion of 
### the reciprocal space, so we need to symmetrize the diffraction data array

data_set = interped.copy()

data_set = data_set[fi(H,0):fi(H,4), fi(K,0):fi(K,4), fi(L,0):]

H_new = H[fi(H,0):fi(H,4)]
K_new = K[fi(K,0):fi(K,4)]
L_new = L[fi(L,0):]

H_new = np.concatenate((-H_new[::-1], H_new))
K_new = np.concatenate((-K_new[::-1], K_new))
L_new = np.concatenate((-L_new[::-1], L_new))

data_set_flip_L = data_set[:,:,::-1]
data_set_total = np.concatenate(( data_set_flip_L, data_set), axis=2)

data_set_flip_H = data_set_total[::-1,:,:]
data_set_total = np.concatenate(( data_set_flip_H, data_set_total), axis=0)


data_set_flip_K = data_set_total[:,::-1,:]
data_set_total = np.concatenate(( data_set_flip_K, data_set_total), axis=1)

print(H_new.shape, K_new.shape, L_new.shape)
print(data_set_total.shape)

data_set_total[np.where(data_set_total==0)] =2 # make sure the values are not 0




In [ ]:
### visualize the symmetrized diffraction pattern
%matplotlib notebook
plt.imshow(np.rot90(np.log(data_set_total[:,400,:])),  aspect='auto')
plt.colorbar()


In [ ]:
### perform the FFT of the symmetric diffraction data array. The array must be symmetric if the FFT is to be real valued only in accordance with Friedel's Law
%matplotlib notebook
import scipy
from scipy.fft import fft, fftfreq
from numpy.fft import fft2, ifft2, fftshift, ifftshift, fftn, ifftn

len(H_new)
len(K_new)
len(L_new)

test_array = data_set_total[400-400:400+400,400-400:400+400,423-400:423+400]
print(test_array.shape)
## include FFT shift
patterson_of_shifted = fftn(fftshift(test_array)[0:len(test_array[0])-1, 0:len(test_array[1])-1, 0:len(test_array[2])-1])



In [ ]:
### check that FFT (Patterson Function) is real valued
print(np.sum(np.abs(np.real(patterson_of_shifted[:,:,:]))))
print(np.sum(np.abs(np.imag(patterson_of_shifted[:,:,:]))))

In [ ]:
### iFFT shift
patterson = ifftshift(patterson_of_shifted)

In [ ]:
### save the 3D PDF as a numpy array
with open('/data3/2021/2021_10_CHESS_3088D/ind_jg2364/index_new_nm/patterson.npy', 'wb') as f:
    np.save(f, patterson)